In [ ]:
# MCP Servers

In [31]:
from src.utils import step_printer


In [ ]:
Let's run our first MCP Server. The MCP-Weather container

Run this in a terminal: 

```bash
CONTAINER_NAME=mcp-weather
CONTAINER_IMAGE="mcp-weather"
CONTAINER_TAG="0.1"
REMOTE_REGISTRY="local-registry-quay-local-quay-registry.apps.ocpvdev01.dal13.infra.demo.redhat.com/rhdp/"

podman run -d \
--name $CONTAINER_NAME \
--network=host \
${REMOTE_REGISTRY}${CONTAINER_IMAGE}:latest \
--port 8005  

In [6]:
!curl --max-time 1 http://localhost:8005/sse 2>/dev/null


event: endpoint
data: /messages/?session_id=b66ec1c75aac460bb1fde0d54dbf3d0b



In [39]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")

# For our development purposes, we might want to change different models and test with them, these lines select the first available model in the Llama Stack server. 
SELECTED_MODEL=models[0].identifier
print("--- Selected model: ---")
print(SELECTED_MODEL)


--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
--- Selected model: ---
meta-llama/Llama-3.2-3B-Instruct


In [40]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]

for tools in registered_tools:
    print(tools)


Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', provider_resource_id='web_search', tool_host='distribution', toolgroup_id='builtin::websearch', type='tool', metadata=None)
Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', provider_resource_id='insert_into_memory', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None)
Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', provider_resource_id='knowledge_search', tool_host='distribution', toolgroup_id='builtin::rag'

In [41]:
client.toolgroups.register(
        toolgroup_id="mcp::mcp-weather",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8005/sse"},
    )

In [42]:
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]

for tools in registered_tools:
    print("\n")
    print(tools)



Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', provider_resource_id='web_search', tool_host='distribution', toolgroup_id='builtin::websearch', type='tool', metadata=None)


Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', provider_resource_id='insert_into_memory', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None)


Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', provider_resource_id='knowledge_search', tool_host='distribution', toolgroup_id='builtin

In [43]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    instructions="""You are a helpful agent with access to tools, use the weather tool to answer questions
            """ ,
    tools=["mcp::mcp-weather"],
)

In [44]:
stream=False
user_prompts = [
       "what is the weather in boulder colorado",
       "are there any weather alerts for Boston at the moment?",
]


for prompt in user_prompts:
    # Generate a new Unique Identifier for each session 
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"web-session-{new_uuid}")

    cprint(f"\n{'='*100}\nProcessing user query: {prompt}\n{'='*100}", "blue")

    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: what is the weather in boulder colorado

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_forecast, Arguments: {'latitude': '40.0116', 'longitude': '-105.2729'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'type': 'text',
│   'text': '\nThis Afternoon:\nTemperature: 67°F\nWind: 6 mph ESE\nForecast: A slight chance of showers and thunderstorms. Mostly sunny. High near 67, with temperatures falling to around 65 in the afternoon. East southeast wind around 6 mph. Chance of precipitation is 20%.\n        \n---\n\nTonight:\nTemperature: 47°F\nWind: 5 mph WSW\nForecast: A slight chance of rain showers before 7pm. Partly cloudy, with a low around 47. West southwest wind around 5 mph. Chance of precipitation is 20%.\n        \n---\n\nFriday:\nTemperature: 72°F\nWind: 2 to 6 mph NNE\nForecast: A slight chance of showers and thunderstorms after noon. Mostly sunny. High near 72, with temperatures falling to around 70 in the afternoon. North northeast wind 2 to 6 mph. Chance of precipitation is 20%.\n        \n---\n\nFriday Night:\nTemperature: 47°F\nWind: 5 mph NW\nForecast: Partly cloudy, with a low around 47. Northwest wind around 5 mph.\n        \n---\n\nSaturday:\nTemperature: 78°F\nWind: 2 to 7 mph SSE\nForecast: Sunny, with a high near 78. South southeast wind 2 to 7 mph.\n        ',
│   'annotations': None
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The current weather in Boulder, Colorado is mostly sunny with a high of 67°F and a low of 47°F. There's a slight chance of showers and thunderstorms this afternoon.

========== Query processing completed ========== 


Processing user query: are there any weather alerts for Boston at the moment?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_alerts, Arguments: {'state': 'MA'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'type': 'text',
│   'text': '\nEvent: Flood Watch\nArea: Hartford; Tolland; Western Franklin; Eastern Franklin; Western Hampshire; Western Hampden; Eastern Hampshire; Eastern Hampden\nSeverity: Severe\nDescription: * WHAT...Flooding caused by excessive rainfall continues to be\npossible.\n\n* WHERE...Portions of northern Connecticut, including the following\nareas, Hartford CT and Tolland CT and western Massachusetts,\nincluding the following areas, Eastern Franklin MA, Eastern\nHampden MA, Eastern Hampshire MA, Western Franklin MA, Western\nHampden MA and Western Hampshire MA.\n\n* WHEN...From late tonight through Saturday morning.\n\n* IMPACTS...Excessive runoff may result in flooding of rivers,\ncreeks, streams, and other low-lying and flood-prone locations.\n\n* ADDITIONAL DETAILS...\n- Moderate to heavy rains expected Friday morning through\nSaturday morning with 2-3 inches of rain possible. Given the\nwet antecedent conditions and already high flows, continued\nriver flooding expected along the southern CT river, and\npossibly into the Northern CT river.  Minor street and\nnuisance flooding is possible, but flash flooding is\nunlikely.\n- http://www.weather.gov/safety/flood\nInstructions: You should monitor later forecasts and be alert for possible Flood\nWarnings. Those living in areas prone to flooding should be prepared\nto take action should flooding develop.\n    \n---\n\nEvent: Flood Watch\nArea: Northern Litchfield; Southern Litchfield; Northern Berkshire; Southern Berkshire; Southern Fulton; Montgomery; Northern Saratoga; Schoharie; Western Schenectady; Eastern Schenectady; Southern Saratoga; Western Albany; Eastern Albany; Western Rensselaer; Eastern Rensselaer; Western Greene; Eastern Greene; Western Columbia; Eastern Columbia; Western Ulster; Eastern Ulster; Western Dutchess; Eastern Dutchess; Northern Fulton; Southeast Warren; Southern Washington; Bennington; Western Windham; Eastern Windham\nSeverity: Severe\nDescription: * WHAT...Flooding caused by excessive rainfall continues to be\npossible.\n\n* WHERE...Portions of northwestern Connecticut, including the\nfollowing areas, Litchfield County, western Massachusetts,\nincluding Berkshire County, in eastern New York, including the\neastern Catskills, mid Hudson Valley, the Greater Capital Region,\ncentral and eastern Mohawk Valley, the Taconics, Schoharie Valley,\nHelderbergs and southern reaches of the Lake George and northern\nSaratoga Region, as well as southern Vermont including Bennington\nand Windham Counties.\n\n* WHEN...From late tonight through Saturday morning.\n\n* IMPACTS...Excessive runoff may result in flooding of rivers,\ncreeks, streams, and other low-lying and flood-prone locations.\nCreeks and streams may rise out of their banks. Flooding may occur\nin poor drainage and urban areas.\n\n* ADDITIONAL DETAILS...\n- A low pressure system will bring 1.5 to 3 inches of rainfall\nto the region late tonight into Saturday morning. This\nrainfall will fall on already saturated soils from heavy\nrainfall from the past weekend into early this week.  Some\nisolated flash flooding can not be ruled out.  Minor flooding\nof main stem rivers, creeks and small streams may occur.\nSome rivers that could reach the minor flood stage include\nthe Esopus, Hoosic, and Housatonic.\n- http://www.weather.gov/safety/flood\nInstructions: You should monitor later forecasts and be alert for possible Flood\nWarnings. Those living in areas prone to flooding should be prepared\nto take action should flooding develop.\n    ',
│   'annotations': None
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
There is a Flood Watch for Boston, which includes the following areas:

* Hartford
* Tolland
* Western Franklin
* Eastern Franklin
* Western Hampshire
* Western Hampden
* Eastern Hampshire
* Eastern Hampden

The flood watch is in effect from late tonight through Saturday morning. Excessive runoff may result in flooding of rivers, creeks, streams, and other low-lying and flood-prone locations. Minor street and nuisance flooding is possible, but flash flooding is unlikely.

========== Query processing completed ========== 

